# Analyzing models

In [ ]:
import escher
from cobra.io import read_sbml_model
from cobra.flux_analysis import flux_variability_analysis
model = read_sbml_model('data/e_coli_core.xml.gz')

In [ ]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

## Flux variablity analysis (Part 1)

Calculate all flux ranges of all reactions in the model.

In [ ]:
result = flux_variability_analysis(model)

Inspect the result.

In [ ]:
result

Get an overview of a few key statistics of the resulting flux ranges.

In [ ]:
result.describe()

Visualize the flux ranges on a pathway map of _E. coli's_ central carbon metabolism.

In [ ]:
abs_flux_ranges = abs(result.maximum - result.minimum).to_dict()
escher.Builder('e_coli_core.Core metabolism', reaction_data=abs_flux_ranges)

Those reactions showing up in red are futile cyles.

In [ ]:
result[result.maximum > 500]

In [ ]:
result_no_cyles = flux_variability_analysis(model, loopless=True)

In [ ]:
abs_flux_ranges = abs(result_no_cyles.maximum - result_no_cyles.minimum).to_dict()
escher.Builder('e_coli_core.Core metabolism', reaction_data=abs_flux_ranges)

## Run flux variability analysis for optimally growing _E. coli_

(Optimal) Flux Balance Analysis solutions are not necessariliy unique. Flux Variablity Analysis is a good tool for estimating the space of alternative optimal solutions.

In [ ]:
fba_solution = model.optimize()

In [ ]:
fba_solution.objective_value

In [ ]:
model_optimal = model.copy()

In [ ]:
model_optimal.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = fba_solution.objective_value

In [ ]:
result_max_obj = flux_variability_analysis(model_optimal, loopless=True)

In [ ]:
result_max_obj

This is actually such a common task that `flux_variability_analysis` provides an option for fixing the objective's flux at a certain percentage.

In [ ]:
result_max_obj = flux_variability_analysis(model, fraction_of_optimum=1., loopless=True)

In [ ]:
result_max_obj

Turns out that in this small core metabolic model, the optimal solution is actually unique!

In [ ]:
sum(abs(result_max_obj.minimum - result_max_obj.maximum))

## Exercises (20 min)

### Exercise 1

Explore how relaxing the constraint on the growth rate affects the solution space:
1. Modify the code to explore flux ranges for $\mu \gt 0.7 \ h^{-1}$ 
1. Plot the sum of flux ranges over a range of percentages.

### Exercise 2

Using FVA, determine all blocked reactions ($v = 0$) in the model.

## Phenotypic phase plane analysis (Part 2)

Load a few packages. Cobrapy implements phenotypic phase plane calculation as well but the cameo version comes with better plotting capabilities.

In [ ]:
import pandas
pandas.options.display.max_rows = 12
from cameo.flux_analysis import phenotypic_phase_plane

Compute the phenotypic phase plane for growth and acetate secretion.

In [ ]:
result = phenotypic_phase_plane(model,
                                variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                objective=model.reactions.EX_ac_e)

Look at the result in a tabular view.

In [ ]:
result.data_frame

Plot the phenotypic phase plane showing the flux through the objective.

In [ ]:
result.plot()

We may also be interested in other parameters such as the carbon yield for output versus input at different growth rates.

In [ ]:
result.plot(estimate='c_yield')

We can also calculate a three dimensional phenotypic phase plane to compare the influence of aerobic and anaerobic environements.

In [ ]:
result_3D = phenotypic_phase_plane(model, variables=[model.reactions.EX_ac_e, model.reactions.EX_o2_e],
                                   objective=model.reactions.BIOMASS_Ecoli_core_w_GAM, points=50)

In [ ]:
result_3D.data_frame

In [ ]:
result_3D.plot()

## Exercise (10 min)

* Use `phenotypic_phase_plane` to determine the optimal O2 uptake rate.